Procedo a analizar la relacion entre el codigo burocratico y si la operacion
es exitosa o no en base al porcentaje de closed Won sobre closed Lost.

In [22]:
import pandas as pd
import seaborn as sb

data = pd.read_csv('Entrenamieto_ECI_2020.csv')
bureaucraticCode = data[['Bureaucratic_Code', 'Stage']]
bureaucraticCode = bureaucraticCode[bureaucraticCode['Stage'].isin(values=['Closed Won', 'Closed Lost'])]
bureaucraticCode.groupby('Bureaucratic_Code').agg()
#sb.countplot(x='Bureaucratic_Code', data=bureaucraticCode)

,Bureaucratic_Code,Stage
0,Bureaucratic_Code_4,Closed Lost
1,Bureaucratic_Code_4,Closed Won
2,Bureaucratic_Code_4,Closed Won
3,Bureaucratic_Code_5,Closed Lost
4,Bureaucratic_Code_5,Closed Lost
...,...,...
16942,Bureaucratic_Code_4,Closed Won
16943,Bureaucratic_Code_4,Closed Won
16944,Bureaucratic_Code_4,Closed Won
16945,Bureaucratic_Code_4,Closed Lost
